# Minority Rule data preparation
Joining Census Bureau demographic data with election data for future mapping.

TODO:
 - handle territories
 - clean up race category headers and include them

In [337]:
import pandas as pd

## Counties
### County Demographics
Source: [2020 Census redistricting data](https://www.census.gov/programs-surveys/decennial-census/about/rdo/summary-files.html)


In [338]:
abbr_df = pd.read_csv('abbr.tsv', delimiter='\t', usecols=['State', 'Code'], index_col='Code')
abbr_lookup = abbr_df.to_dict()['State']
abbr_lookup

{'AL': 'Alabama',
 'AK': 'Alaska',
 'AZ': 'Arizona',
 'AR': 'Arkansas',
 'CA': 'California',
 'CO': 'Colorado',
 'CT': 'Connecticut',
 'DE': 'Delaware',
 'DC': 'District of Columbia',
 'FL': 'Florida',
 'GA': 'Georgia',
 'HI': 'Hawaii',
 'ID': 'Idaho',
 'IL': 'Illinois',
 'IN': 'Indiana',
 'IA': 'Iowa',
 'KS': 'Kansas',
 'KY': 'Kentucky',
 'LA': 'Louisiana',
 'ME': 'Maine',
 'MD': 'Maryland',
 'MA': 'Massachusetts',
 'MI': 'Michigan',
 'MN': 'Minnesota',
 'MS': 'Mississippi',
 'MO': 'Missouri',
 'MT': 'Montana',
 'NE': 'Nebraska',
 'NV': 'Nevada',
 'NH': 'New Hampshire',
 'NJ': 'New Jersey',
 'NM': 'New Mexico',
 'NY': 'New York',
 'NC': 'North Carolina',
 'ND': 'North Dakota',
 'OH': 'Ohio',
 'OK': 'Oklahoma',
 'OR': 'Oregon',
 'PA': 'Pennsylvania',
 'RI': 'Rhode Island',
 'SC': 'South Carolina',
 'SD': 'South Dakota',
 'TN': 'Tennessee',
 'TX': 'Texas',
 'UT': 'Utah',
 'VT': 'Vermont',
 'VA': 'Virginia',
 'WA': 'Washington',
 'WV': 'West Virginia',
 'WI': 'Wisconsin',
 'WY': 'Wyoming

In [339]:
# FIPS codes and county names to add FIPS code to demog data later on
# https://www.nrcs.usda.gov/wps/portal/nrcs/detail/national/home/?cid=nrcs143_013697

fips_df = pd.read_csv('county_fips.csv', dtype={'FIPS': str})
# todo: have the lookup handle territories - currently they'll have NaNs
fips_df['state_name'] = fips_df['State'].map(abbr_lookup)
fips_df['full_name'] = fips_df['Name'] + ', ' + fips_df['state_name']
fips_df = fips_df.set_index('full_name')
# fips_df.to_csv('county_fips.csv')
fips_df

,FIPS,Name,State,state_name
full_name,,,,
"Autauga, Alabama",01001,Autauga,AL,Alabama
"Baldwin, Alabama",01003,Baldwin,AL,Alabama
"Barbour, Alabama",01005,Barbour,AL,Alabama
"Bibb, Alabama",01007,Bibb,AL,Alabama
"Blount, Alabama",01009,Blount,AL,Alabama
...,...,...,...,...
NaN,72151,Yabucoa,PR,NaN
NaN,72153,Yauco,PR,NaN
NaN,78010,St. Croix,VI,NaN


In [340]:
# census 2020 redistricting data
county_demog_df = pd.read_csv(
    'county-demog.csv',
    index_col='Label (Grouping)',
    skip_blank_lines=True,
    thousands=','
).transpose()
county_demog_df.index.name = 'county'
# remove county and parish to standardize across states
county_demog_df.index = county_demog_df.index.map(lambda x: x.replace(' County,', ',').replace(' Parish,', ','))
# combine demographic data with FIPS codes
county_demog_df = county_demog_df.join(fips_df).set_index('FIPS')
# limit demographics to only total pop for now
county_demog_df = county_demog_df[['Total:', 'Name', 'State', 'state_name']]
county_demog_df = county_demog_df.rename(
    {'Total:': 'total_pop', 'Name': 'name', 'State': 'state'}
)
county_demog_df

,Total:,Name,State,state_name
FIPS,,,,
45001,24295,Abbeville,SC,South Carolina
22001,57576,Acadia,LA,Louisiana
51001,33413,Accomack,VA,Virginia
16001,494967,Ada,ID,Idaho
19001,7496,Adair,IA,Iowa
...,...,...,...,...
04027,203881,Yuma,AZ,Arizona
08125,9988,Yuma,CO,Colorado
48505,13889,Zapata,TX,Texas


### County 2020 Presidential Election Results
Source: [County Presidential Election Returns 2000-2020](https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/VOQCHQ)

Dataset:
```
MIT Election Data and Science Lab, 2018, "County Presidential Election Returns 2000-2020",
 https://doi.org/10.7910/DVN/VOQCHQ, Harvard Dataverse, V10,
 UNF:6:pVAMya52q7VM1Pl7EZMW0Q== [fileUNF]
```

File:
```
MIT Election Data and Science Lab, 2018, "County Presidential Election Returns 2000-2020",
 https://doi.org/10.7910/DVN/VOQCHQ, Harvard Dataverse, V10;
 countypres_2000-2020.tab [fileName], UNF:6:pVAMya52q7VM1Pl7EZMW0Q== [fileUNF]
```

In [341]:
county_pres_df = pd.read_csv(
    'countypres_2000-2020.tab',
    index_col=['county_fips'],
    delimiter='\t',
    dtype={'county_fips': str, 'version': str}
)

# limit to major parties for now
county_pres_df = county_pres_df[
    (county_pres_df['year'] == 2020) &
    (county_pres_df['party'].isin(('DEMOCRAT', 'REPUBLICAN')))
    ]

total_series = county_pres_df['totalvotes'].groupby(county_pres_df.index).max()

county_pres_df = county_pres_df.pivot_table(
    index=county_pres_df.index,
    columns='party',
    values='candidatevotes'
)
county_pres_df['TOTAL'] = total_series

county_pres_df['dem_percent'] = 100 * county_pres_df['DEMOCRAT'] / county_pres_df['TOTAL']
county_pres_df['rep_percent'] = 100 * county_pres_df['REPUBLICAN'] / county_pres_df['TOTAL']
county_pres_df['dem_lead'] = 100 * county_pres_df['DEMOCRAT'] - county_pres_df['REPUBLICAN']
county_pres_df['dem_percent_lead'] = county_pres_df['dem_percent'] - county_pres_df['rep_percent']

county_pres_df.to_csv('county_results.csv')
county_pres_df

party,DEMOCRAT,REPUBLICAN,TOTAL,dem_percent,rep_percent,dem_lead,dem_percent_lead
county_fips,,,,,,,
01001,7503.0,19838.0,27770,27.018365,71.436802,730462.0,-44.418437
01003,24578.0,83544.0,109679,22.409030,76.171373,2374256.0,-53.762343
01005,4816.0,5622.0,10518,45.788173,53.451226,475978.0,-7.663054
01007,1986.0,7525.0,9595,20.698280,78.426264,191075.0,-57.727983
01009,2640.0,24711.0,27588,9.569378,89.571553,239289.0,-80.002175
...,...,...,...,...,...,...,...
56037,3823.0,12229.0,16698,22.894957,73.236316,370071.0,-50.341358
56039,9848.0,4341.0,14787,66.599040,29.356868,980459.0,37.242172
56041,1591.0,7496.0,9459,16.819960,79.247278,151604.0,-62.427318


### Combine to one table

In [342]:
county_full_df = county_pres_df.join(county_demog_df)
county_full_df.index.name = 'fips'
county_full_df = county_full_df.rename(columns=str.lower)
county_full_df.to_csv('county-combined.csv')
county_full_df

,democrat,republican,total,dem_percent,rep_percent,dem_lead,dem_percent_lead,total:,name,state,state_name
fips,,,,,,,,,,,
01001,7503.0,19838.0,27770,27.018365,71.436802,730462.0,-44.418437,58805.0,Autauga,AL,Alabama
01003,24578.0,83544.0,109679,22.409030,76.171373,2374256.0,-53.762343,231767.0,Baldwin,AL,Alabama
01005,4816.0,5622.0,10518,45.788173,53.451226,475978.0,-7.663054,25223.0,Barbour,AL,Alabama
01007,1986.0,7525.0,9595,20.698280,78.426264,191075.0,-57.727983,22293.0,Bibb,AL,Alabama
01009,2640.0,24711.0,27588,9.569378,89.571553,239289.0,-80.002175,59134.0,Blount,AL,Alabama
...,...,...,...,...,...,...,...,...,...,...,...
56037,3823.0,12229.0,16698,22.894957,73.236316,370071.0,-50.341358,42272.0,Sweetwater,WY,Wyoming
56039,9848.0,4341.0,14787,66.599040,29.356868,980459.0,37.242172,23331.0,Teton,WY,Wyoming
56041,1591.0,7496.0,9459,16.819960,79.247278,151604.0,-62.427318,20450.0,Uinta,WY,Wyoming


## States
### State Demographics
Source: [2020 Census redistricting data](https://www.census.gov/programs-surveys/decennial-census/about/rdo/summary-files.html)



In [343]:
state_demog_df = pd.read_csv(
    'state-demog.csv',
    index_col='Label (Grouping)',
    skip_blank_lines=True,
    thousands=','
).transpose()

state_demog_df.index.name = 'state'
state_demog_df.index = state_demog_df.index.str.upper()

# limit demographics to only total pop for now
state_demog_df = state_demog_df[['Total:']]
state_demog_df = state_demog_df.rename(columns={'Total:': 'total_pop'})

state_demog_df

Label (Grouping),total_pop
state,
ALABAMA,5024279
ALASKA,733391
ARIZONA,7151502
ARKANSAS,3011524
CALIFORNIA,39538223
COLORADO,5773714
CONNECTICUT,3605944
DELAWARE,989948
DISTRICT OF COLUMBIA,689545


## State 2020 Presidential Election Results
Source: [U.S. President 1976–2020](https://dataverse.harvard.edu/file.xhtml?fileId=4299753&version=6.0)

Dataset
```
 MIT Election Data and Science Lab, 2017, "U.S. President 1976–2020",
  https://doi.org/10.7910/DVN/42MVDX, Harvard Dataverse, V6,
  UNF:6:4KoNz9KgTkXy0ZBxJ9ZkOw== [fileUNF]
```

File
```
 MIT Election Data and Science Lab, 2017, "U.S. President 1976–2020",
  https://doi.org/10.7910/DVN/42MVDX, Harvard Dataverse, V6;
  1976-2020-president.tab [fileName], UNF:6:4KoNz9KgTkXy0ZBxJ9ZkOw== [fileUNF]
```


In [344]:
state_elect_df = pd.read_csv('1976-2020-president.tab', delimiter='\t', index_col='state')
state_elect_df = state_elect_df[
    (state_elect_df['year'] == 2020) & (state_elect_df['party_simplified'].isin(['DEMOCRAT', 'REPUBLICAN']))]

total_series = state_elect_df['totalvotes'].groupby(state_elect_df.index).max()

state_elect_df = state_elect_df.pivot_table(index=state_elect_df.index, columns='party_simplified',
                                            values='candidatevotes')
state_elect_df['TOTAL'] = total_series
state_elect_df['dem_percent'] = 100 * state_elect_df['DEMOCRAT'] / state_elect_df['TOTAL']
state_elect_df['rep_percent'] = 100 * state_elect_df['REPUBLICAN'] / state_elect_df['TOTAL']
state_elect_df['dem_lead'] = 100 * state_elect_df['DEMOCRAT'] - state_elect_df['REPUBLICAN']
state_elect_df['dem_percent_lead'] = state_elect_df['dem_percent'] - state_elect_df['rep_percent']
state_elect_df.to_csv('state_results.csv')
state_elect_df

party_simplified,DEMOCRAT,REPUBLICAN,TOTAL,dem_percent,rep_percent,dem_lead,dem_percent_lead
state,,,,,,,
ALABAMA,849624,1441170,2323282,36.569990,62.031643,83521230,-25.461653
ALASKA,153778,189951,359530,42.771952,52.833143,15187849,-10.061191
ARIZONA,1672143,1661686,3387326,49.364691,49.055981,165552614,0.308710
ARKANSAS,423932,760647,1219069,34.775062,62.395730,41632553,-27.620668
CALIFORNIA,11110250,6006429,17500881,63.483947,34.320724,1105018571,29.163223
COLORADO,1804352,1364607,3279980,55.011067,41.604126,179070593,13.406942
CONNECTICUT,1080831,714717,1823857,59.260732,39.187118,107368383,20.073613
DELAWARE,296268,200603,504346,58.743006,39.774877,29426197,18.968129
DISTRICT OF COLUMBIA,317323,18586,344356,92.149694,5.397321,31713714,86.752373


### Combine to one table

In [345]:
state_full_df = state_elect_df.join(state_demog_df)
state_full_df.index.name = 'state'
state_full_df = state_full_df.rename(columns=str.lower)
state_full_df.to_csv('state-combined.csv')
state_full_df

,democrat,republican,total,dem_percent,rep_percent,dem_lead,dem_percent_lead,total_pop
state,,,,,,,,
ALABAMA,849624,1441170,2323282,36.569990,62.031643,83521230,-25.461653,5024279
ALASKA,153778,189951,359530,42.771952,52.833143,15187849,-10.061191,733391
ARIZONA,1672143,1661686,3387326,49.364691,49.055981,165552614,0.308710,7151502
ARKANSAS,423932,760647,1219069,34.775062,62.395730,41632553,-27.620668,3011524
CALIFORNIA,11110250,6006429,17500881,63.483947,34.320724,1105018571,29.163223,39538223
COLORADO,1804352,1364607,3279980,55.011067,41.604126,179070593,13.406942,5773714
CONNECTICUT,1080831,714717,1823857,59.260732,39.187118,107368383,20.073613,3605944
DELAWARE,296268,200603,504346,58.743006,39.774877,29426197,18.968129,989948
DISTRICT OF COLUMBIA,317323,18586,344356,92.149694,5.397321,31713714,86.752373,689545


## Create GeoJSONs
Source: [US Census Bureau Cartographic Boundary Files](https://www.census.gov/geographies/mapping-files/time-series/geo/carto-boundary-file.html)

In [346]:
import geopandas as gpd

county_geo_df = gpd.read_file('cb_2018_us_county_20m/cb_2018_us_county_20m.shp')
county_geo_df['NAME'] = county_geo_df['NAME'].str.upper()
county_geo_df.set_index('NAME')
county_geo_df = county_geo_df.join(state_full_df, on='NAME')
county_geo_df.to_file('counties.geojson', driver='GeoJSON')
county_geo_df

/Users/steve/notebooks/venv/lib/python3.9/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


,STATEFP,COUNTYFP,COUNTYNS,AFFGEOID,GEOID,NAME,LSAD,ALAND,AWATER,geometry,democrat,republican,total,dem_percent,rep_percent,dem_lead,dem_percent_lead,total_pop
0,37,017,01026336,0500000US37017,37017,BLADEN,06,2265887723,33010866,"POLYGON ((-78.90200 34.83527, -78.79960 34.850...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,37,167,01025844,0500000US37167,37167,STANLY,06,1023370459,25242751,"POLYGON ((-80.49737 35.20210, -80.29542 35.502...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,39,153,01074088,0500000US39153,39153,SUMMIT,06,1069181981,18958267,"POLYGON ((-81.68699 41.13596, -81.68495 41.277...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,42,113,01213687,0500000US42113,42113,SULLIVAN,06,1165338428,6617028,"POLYGON ((-76.81373 41.59003, -76.22014 41.541...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,48,459,01384015,0500000US48459,48459,UPSHUR,06,1509910100,24878888,"POLYGON ((-95.15274 32.66095, -95.15211 32.902...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3215,38,005,01034216,0500000US38005,38005,BENSON,06,3596569006,131708143,"POLYGON ((-99.84661 48.37130, -99.49292 48.370...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3216,72,079,01804520,0500000US72079,72079,LAJAS,13,155287827,106643202,"POLYGON ((-67.10904 18.05608, -67.06791 18.066...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3217,31,159,00835901,0500000US31159,31159,SEWARD,06,1479995670,11542537,"POLYGON ((-97.36812 41.04695, -96.91094 41.046...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3218,37,023,01008539,0500000US37023,37023,BURKE,06,1311146878,20719896,"POLYGON ((-81.90665 35.88338, -81.94319 35.960...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [347]:
state_geo_df = gpd.read_file('cb_2018_us_state_20m/cb_2018_us_state_20m.shp')
state_geo_df['NAME'] = state_geo_df['NAME'].str.upper()
state_geo_df.set_index('NAME')
state_geo_df = state_geo_df.join(state_full_df, on='NAME')
state_geo_df.to_file('states.geojson', driver='GeoJSON')
state_geo_df

/Users/steve/notebooks/venv/lib/python3.9/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


,STATEFP,STATENS,AFFGEOID,GEOID,STUSPS,NAME,LSAD,ALAND,AWATER,geometry,democrat,republican,total,dem_percent,rep_percent,dem_lead,dem_percent_lead,total_pop
0,24,01714934,0400000US24,24,MD,MARYLAND,00,25151100280,6979966958,"MULTIPOLYGON (((-76.04621 38.02553, -76.00734 ...",1985023.0,976414.0,3037030.0,65.360665,32.150292,1.975259e+08,33.210373,6177224.0
1,19,01779785,0400000US19,19,IA,IOWA,00,144661267977,1084180812,"POLYGON ((-96.62187 42.77925, -96.57794 42.827...",759061.0,897672.0,1700130.0,44.647233,52.800198,7.500843e+07,-8.152965,3190369.0
2,10,01779781,0400000US10,10,DE,DELAWARE,00,5045925646,1399985648,"POLYGON ((-75.77379 39.72220, -75.75323 39.757...",296268.0,200603.0,504346.0,58.743006,39.774877,2.942620e+07,18.968129,989948.0
3,39,01085497,0400000US39,39,OH,OHIO,00,105828882568,10268850702,"MULTIPOLYGON (((-82.86334 41.69369, -82.82572 ...",2679165.0,3154834.0,5922202.0,45.239338,53.271300,2.647617e+08,-8.031962,11799448.0
4,42,01779798,0400000US42,42,PA,PENNSYLVANIA,00,115884442321,3394589990,"POLYGON ((-80.51989 40.90666, -80.51964 40.987...",3458229.0,3377674.0,6915283.0,50.008496,48.843612,3.424452e+08,1.164884,13002700.0
5,31,01779792,0400000US31,31,NE,NEBRASKA,00,198956658395,1371829134,"POLYGON ((-104.05314 41.11446, -104.05245 41.2...",374583.0,556846.0,956383.0,39.166631,58.224163,3.690145e+07,-19.057532,1961504.0
6,53,01779804,0400000US53,53,WA,WASHINGTON,00,172112588220,12559278850,"MULTIPOLYGON (((-123.23715 48.68347, -123.0704...",2369612.0,1584651.0,4087631.0,57.970301,38.766978,2.353765e+08,19.203323,7705281.0
7,72,01779808,0400000US72,72,PR,PUERTO RICO,00,8868896030,4922382562,"MULTIPOLYGON (((-65.34207 18.34529, -65.25593 ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,01,01779775,0400000US01,01,AL,ALABAMA,00,131174048583,4593327154,"POLYGON ((-88.46866 31.89386, -88.46866 31.933...",849624.0,1441170.0,2323282.0,36.569990,62.031643,8.352123e+07,-25.461653,5024279.0
9,05,00068085,0400000US05,05,AR,ARKANSAS,00,134768872727,2962859592,"POLYGON ((-94.61792 36.49941, -94.36120 36.499...",423932.0,760647.0,1219069.0,34.775062,62.395730,4.163255e+07,-27.620668,3011524.0
